In [114]:
import os

In [115]:
%pwd

'C:\\Users\\KIIT\\Desktop\\CODING\\chest_cancer_using mlflow\\endtoend-chest_cancer-proj'

In [116]:
%cd "C:\Users\KIIT\Desktop\CODING\chest_cancer_using mlflow\endtoend-chest_cancer-proj"

C:\Users\KIIT\Desktop\CODING\chest_cancer_using mlflow\endtoend-chest_cancer-proj


In [117]:
%ls

 Volume in drive C is Windows-SSD
 Volume Serial Number is 2CD7-88B3

 Directory of C:\Users\KIIT\Desktop\CODING\chest_cancer_using mlflow\endtoend-chest_cancer-proj

01-07-2024  01:49    <DIR>          .
27-06-2024  13:50    <DIR>          ..
27-06-2024  14:38    <DIR>          .github
01-07-2024  01:51             3,316 .gitignore
08-07-2024  19:39    <DIR>          artifacts
27-06-2024  14:38    <DIR>          config
30-06-2024  20:26                 0 dvc.yaml
27-06-2024  21:17             1,093 LICENSE
27-06-2024  18:06    <DIR>          logs
01-07-2024  01:47               436 main.py
08-07-2024  19:20               144 params.yaml
30-06-2024  21:05               294 README.md
27-06-2024  15:35               193 requirements.txt
03-07-2024  17:24    <DIR>          research
30-06-2024  20:25               808 setup.py
27-06-2024  15:37    <DIR>          src
27-06-2024  14:38             1,250 template.py
27-06-2024  14:38    <DIR>          templates
               9 File(s)       

In [118]:
%pwd

'C:\\Users\\KIIT\\Desktop\\CODING\\chest_cancer_using mlflow\\endtoend-chest_cancer-proj'

In [119]:
import os

print("Current Working Directory:", os.getcwd())


Current Working Directory: C:\Users\KIIT\Desktop\CODING\chest_cancer_using mlflow\endtoend-chest_cancer-proj


In [120]:
from pathlib import Path

CONFIG_FILE_PATH = Path('config/config.yaml')
PARAMS_FILE_PATH = Path('params.yaml')

print(f"Absolute Config Path: {CONFIG_FILE_PATH.resolve()}")
print(f"Absolute Params Path: {PARAMS_FILE_PATH.resolve()}")

print(f"Config path exists: {CONFIG_FILE_PATH.exists()}")
print(f"Params path exists: {PARAMS_FILE_PATH.exists()}")


Absolute Config Path: C:\Users\KIIT\Desktop\CODING\chest_cancer_using mlflow\endtoend-chest_cancer-proj\config\config.yaml
Absolute Params Path: C:\Users\KIIT\Desktop\CODING\chest_cancer_using mlflow\endtoend-chest_cancer-proj\params.yaml
Config path exists: True
Params path exists: True


In [121]:
from pathlib import Path
import os

# Assuming the script is in the root directory of the project
CONFIG_FILE_PATH = Path('config/config.yaml')
PARAMS_FILE_PATH = Path('params.yaml')

print(f"Absolute Config Path: {CONFIG_FILE_PATH.resolve()}")
print(f"Absolute Params Path: {PARAMS_FILE_PATH.resolve()}")


Absolute Config Path: C:\Users\KIIT\Desktop\CODING\chest_cancer_using mlflow\endtoend-chest_cancer-proj\config\config.yaml
Absolute Params Path: C:\Users\KIIT\Desktop\CODING\chest_cancer_using mlflow\endtoend-chest_cancer-proj\params.yaml


In [122]:

print(f"Config path exists: {CONFIG_FILE_PATH.exists()}")
print(f"Params path exists: {PARAMS_FILE_PATH.exists()}")


Config path exists: True
Params path exists: True


In [138]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    

In [130]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import create_directories,read_yaml


In [139]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [140]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [136]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        try:
         model.save(path)
        except Exception as e:
            print(f"error occured in loading the model{path} due to {e}") 

In [137]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-07-08 20:39:31,935: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-08 20:39:31,941: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-08 20:39:31,944: INFO: common: created directory at: artifacts]
[2024-07-08 20:39:31,944: INFO: common: created directory at: artifacts/prepare_base_model]


TypeError: PrepareBaseModelConfig.__init__() got an unexpected keyword argument 'root_dir'